# Capstone Two: Data Wrangling

Data from: https://www.kaggle.com/c/competitive-data-science-predict-future-sales

The goal is to predict total sales for every product and store in the next month, and potentially get other insights.

In [3]:
# import libraries and format output
import numpy as np
import pandas as pd
import seaborn as sns
import os
from library.sb_utils import save_file
pd.options.display.float_format = '{:.2f}'.format

##### File descriptions
sales_train.csv - the training set. Daily historical data from January 2013 to October 2015.

test.csv - the test set. You need to forecast the sales for these shops and products for November 2015.

sample_submission.csv - a sample submission file in the correct format.

items.csv - supplemental information about the items/products.

item_categories.csv  - supplemental information about the items categories.

shops.csv- supplemental information about the shops.

In [5]:
# get all the csv files from the data folder
data_path = './data/'
sales_train = pd.read_csv(data_path+"sales_train.csv")
test_dataset = pd.read_csv(data_path+"test.csv")
items = pd.read_csv(data_path+"items.csv")
item_categories = pd.read_csv(data_path+"item_categories.csv")
shops = pd.read_csv(data_path+"shops.csv")
#Let's make sure they all loaded
[loaded.columns for loaded in [sales_train, test_dataset, items, item_categories, shops]]

[Index(['date', 'date_block_num', 'shop_id', 'item_id', 'item_price',
        'item_cnt_day'],
       dtype='object'),
 Index(['ID', 'shop_id', 'item_id'], dtype='object'),
 Index(['item_name', 'item_id', 'item_category_id'], dtype='object'),
 Index(['item_category_name', 'item_category_id'], dtype='object'),
 Index(['shop_name', 'shop_id'], dtype='object')]

In [6]:
# Check for null values in our train and test data
print(sales_train.isna().sum())
print(test_dataset.isna().sum())

date              0
date_block_num    0
shop_id           0
item_id           0
item_price        0
item_cnt_day      0
dtype: int64
ID         0
shop_id    0
item_id    0
dtype: int64


In [4]:
# Let's do some basic exploration of the dataframes and find how many records are in each of our dataframes.
print(sales_train.shape)
print(test_dataset.shape)
print(items.shape)
print(item_categories.shape)
print(shops.shape)

(2935849, 6)
(214200, 3)
(22170, 3)
(84, 2)
(60, 2)


In [5]:
# Since it is clear that there are more items than item categories and shops, 
# and that our train set is larget than our test set, this makes sense. 
# Let's continue to look at our training data 
sales_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.00
1,03.01.2013,0,25,2552,899.00,1.00
2,05.01.2013,0,25,2552,899.00,-1.00
3,06.01.2013,0,25,2554,1709.05,1.00
4,15.01.2013,0,25,2555,1099.00,1.00


In [6]:
sales_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935849 entries, 0 to 2935848
Data columns (total 6 columns):
 #   Column          Dtype  
---  ------          -----  
 0   date            object 
 1   date_block_num  int64  
 2   shop_id         int64  
 3   item_id         int64  
 4   item_price      float64
 5   item_cnt_day    float64
dtypes: float64(2), int64(3), object(1)
memory usage: 134.4+ MB


In [7]:
sales_train[sales_train.date_block_num == 0]['date'].head(20)

0     02.01.2013
1     03.01.2013
2     05.01.2013
3     06.01.2013
4     15.01.2013
5     10.01.2013
6     02.01.2013
7     04.01.2013
8     11.01.2013
9     03.01.2013
10    03.01.2013
11    05.01.2013
12    07.01.2013
13    08.01.2013
14    10.01.2013
15    11.01.2013
16    13.01.2013
17    16.01.2013
18    26.01.2013
19    27.01.2013
Name: date, dtype: object

In [8]:
# I want to add a datetime format column for later. The rest of the info looks fine as is. Easy fix!
sales_train['datetime'] = pd.to_datetime(sales_train['date'], format='%d.%m.%Y')
sales_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935849 entries, 0 to 2935848
Data columns (total 7 columns):
 #   Column          Dtype         
---  ------          -----         
 0   date            object        
 1   date_block_num  int64         
 2   shop_id         int64         
 3   item_id         int64         
 4   item_price      float64       
 5   item_cnt_day    float64       
 6   datetime        datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(3), object(1)
memory usage: 156.8+ MB


In [9]:
sales_train[sales_train.date_block_num == 0]['date'].head()

0    02.01.2013
1    03.01.2013
2    05.01.2013
3    06.01.2013
4    15.01.2013
Name: date, dtype: object

In [10]:
# Now, let's look at the unique items in each feature. 
# [Index(['date', 'date_block_num', 'shop_id', 'item_id', 'item_price', 'item_cnt_day'],
print('date_block_num')
print(sales_train.sort_values('date_block_num').date_block_num.unique())
print(len(sales_train.sort_values('date_block_num').date_block_num.unique()))

print('\nshop_id')
print(sales_train.sort_values('shop_id').shop_id.unique())
print(len(sales_train.sort_values('shop_id').shop_id.unique()))

print('\nitem_id')
print(sales_train.sort_values('item_id').item_id.unique())
print(len(sales_train.sort_values('item_id').item_id.unique()))

date_block_num
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33]
34

shop_id
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59]
60

item_id
[    0     1     2 ... 22167 22168 22169]
21807


Not every ID is present in the item_id column, which makes sense, but the date_block_num and the shop_id both have the full range without missing values. Nice! This dataset is great. 

What are the minimum and maximum prices in the item_price and item_cnt_day columns? 

In [11]:
print(sales_train.item_price.describe())
print(sales_train.item_cnt_day.describe())

count   2935849.00
mean        890.85
std        1729.80
min          -1.00
25%         249.00
50%         399.00
75%         999.00
max      307980.00
Name: item_price, dtype: float64
count   2935849.00
mean          1.24
std           2.62
min         -22.00
25%           1.00
50%           1.00
75%           1.00
max        2169.00
Name: item_cnt_day, dtype: float64


We have some negative values - in item count, it could represent returns or a canceled sale, but it's impossible to sell an item for a negative price! Let's take a closer look at those columns.

In [12]:
neg_price = sales_train[sales_train['item_price']<=0]
neg_price

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,datetime
484683,15.05.2013,4,32,2973,-1.00,1.00,2013-05-15


Just one value, we could drop that row with the following command. 

```sales_train.drop(neg_price.index, inplace=True)```

However, there could still be valueable info in it from item category or shop information so we'll keep it. 

In [13]:
sales_train[sales_train['item_cnt_day']<=0].describe()

,date_block_num,shop_id,item_id,item_price,item_cnt_day
count,7356.00,7356.00,7356.00,7356.00,7356.00
mean,14.63,32.51,8982.02,1442.95,-1.03
std,9.44,16.83,6166.32,2419.68,0.37
min,0.00,2.00,28.00,0.50,-22.00
25%,7.00,19.00,3734.00,349.00,-1.00
50%,13.00,31.00,6927.00,799.00,-1.00
75%,23.00,47.00,14056.00,1699.00,-1.00
max,33.00,59.00,22167.00,33490.00,-1.00


Seems like almost all of the negative values are -1. If we could reach out to someone to know if this is from missing data or returns, that would be great. Since we can't, I'll keep the data. 

Because of this, I am going to clip the data later in the process, after I have made the monthly features.

Next, I want to check if there are any values in training data that are not in items, item_categories, or shops

In [14]:
# Check if there are any values in training data that are not in items, item_categories, or shops
print(sales_train[~sales_train.item_id.isin(items.item_id)])
print(sales_train[~sales_train.shop_id.isin(shops.shop_id)])
print(items[~items.item_category_id.isin(item_categories.item_category_id)])

Empty DataFrame
Columns: [date, date_block_num, shop_id, item_id, item_price, item_cnt_day, datetime]
Index: []
Empty DataFrame
Columns: [date, date_block_num, shop_id, item_id, item_price, item_cnt_day, datetime]
Index: []
Empty DataFrame
Columns: [item_name, item_id, item_category_id]
Index: []


Every item in our training data has shop, item, and category info! Knowing this, I can confidently join the tables later on.

Now, I want to check if there are any duplicated rows. 

In [15]:
print(sales_train[sales_train.duplicated()])
print(items[items.duplicated()])
print(test_dataset[test_dataset.duplicated()])
print(item_categories[item_categories.duplicated()])
print(shops[shops.duplicated()])

               date  date_block_num  shop_id  item_id  item_price  \
76962    05.01.2013               0       54    20130      149.00   
1435367  23.02.2014              13       50     3423      999.00   
1496766  23.03.2014              14       21     3423      999.00   
1671873  01.05.2014              16       50     3423      999.00   
1866340  12.07.2014              18       25     3423      999.00   
2198566  31.12.2014              23       42    21619      499.00   

         item_cnt_day   datetime  
76962            1.00 2013-01-05  
1435367          1.00 2014-02-23  
1496766          1.00 2014-03-23  
1671873          1.00 2014-05-01  
1866340          1.00 2014-07-12  
2198566          1.00 2014-12-31  
Empty DataFrame
Columns: [item_name, item_id, item_category_id]
Index: []
Empty DataFrame
Columns: [ID, shop_id, item_id]
Index: []
Empty DataFrame
Columns: [item_category_name, item_category_id]
Index: []
Empty DataFrame
Columns: [shop_name, shop_id]
Index: []


The only duplicated data is in the sales dataset; there are no other duplicated rows. 

In [16]:
# finally, we drop duplicates. 
print(sales_train.shape)
sales_train.drop_duplicates(keep='last', inplace=True)
print(sales_train.shape)
sales_train[sales_train.duplicated()]

(2935849, 7)
(2935843, 7)


,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,datetime


Let's look more at the names of items and shops to make sure none of them are duplicates or misspellings. 

In [17]:
print(len(items.item_id.unique()) == len(items.item_name.unique()))
print(len(shops.shop_name.unique()) == len(shops.shop_id.unique()))

True
True


Assuming that the item id and shop id are unique and correct, then we are good to go! 

In [19]:
# Save the data
datapath = './data/clean'
save_file(sales_train, 'clean_sales_train.csv', datapath)
save_file(test_dataset, 'clean_test.csv', datapath)
save_file(items, 'clean_items.csv', datapath)
save_file(item_categories, 'clean_item_categories.csv', datapath)
save_file(shops, 'clean_shops.csv', datapath)

Directory ./data/clean was created.
Writing file.  "./data/clean\clean_sales_train.csv"
Writing file.  "./data/clean\clean_test.csv"
Writing file.  "./data/clean\clean_items.csv"
Writing file.  "./data/clean\clean_item_categories.csv"
Writing file.  "./data/clean\clean_shops.csv"
